In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import re
import zipfile
import io

In [21]:
def simple_get(url):

    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        print('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)

In [112]:
URL= 'https://www.go-hero.net/jam/08/solutions/0/1/Python/'
PATH="Solutions_in_python/"

In [55]:
html_raw=simple_get(URL+"partial/2")

In [27]:
html=BeautifulSoup(html_raw,'html.parser')

In [64]:
link=html.select('a')[0]

In [81]:
def get_download_link(link):
    try:
        download_link= "https://code"+ re.search('"http://code(.*?)"',str(link)).group(1)
        download_link= re.sub("&amp;","&",download_link)
        return download_link
    except AttributeError:
        print('No download-link found')
        return None


In [100]:
text=get_download_link(link)

In [115]:
def download_file_from(link,path=PATH):
    r = get(link,stream=True)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(path=path)
